<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Multi-Query-Retriever

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [1]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

For pip installing chromadb. Refer to https://stackoverflow.com/questions/73969269/error-could-not-build-wheels-for-hnswlib-which-is-required-to-install-pyproject when encountering the error "Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully."

In [2]:
!pip install chromadb

  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.30.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.18.0-cp312-cp312-win_amd64.whl.metadata (4.4 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_

### Loader

Load from wikipedia using wikipedia loader, with known hard limit of query length as 300. One wikipage = 1 Document

In [3]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
len(documents)

24

### Split Documents

In [5]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 525, which is longer than the specified 500


### OpenAI Connection for Embeddings

In [6]:
import os
f = open('../openai_key.txt')
api_key = f.read()
embedding_function = OpenAIEmbeddings(api_key=api_key)

c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
documents

[Document(page_content='Project MKUltra was an illegal human experiments program designed and undertaken by the U.S. Central Intelligence Agency (CIA) to develop procedures and identify drugs that could be used during interrogations to weaken people and force confessions through brainwashing and psychological torture. It began in 1953 and was halted in 1973. MKUltra used numerous methods to manipulate its subjects\' mental states and brain functions, such as the covert administration of high doses of psychoactive drugs (especially LSD) and other chemicals without the subjects\' consent, electroshocks, hypnosis, sensory deprivation, isolation, verbal and sexual abuse, and other forms of torture.\nMKUltra was preceded by Project Artichoke. It was organized through the CIA\'s Office of Scientific Intelligence and coordinated with the United States Army Biological Warfare Laboratories. The program engaged in illegal activities, including the use of U.S. and Canadian citizens as unwitting t

In [8]:
# docs
docs

[Document(page_content="Project MKUltra was an illegal human experiments program designed and undertaken by the U.S. Central Intelligence Agency (CIA) to develop procedures and identify drugs that could be used during interrogations to weaken people and force confessions through brainwashing and psychological torture. It began in 1953 and was halted in 1973. MKUltra used numerous methods to manipulate its subjects' mental states and brain functions, such as the covert administration of high doses of psychoactive drugs (especially LSD) and other chemicals without the subjects' consent, electroshocks, hypnosis, sensory deprivation, isolation, verbal and sexual abuse, and other forms of torture.\nMKUltra was preceded by Project Artichoke. It was organized through the CIA's Office of Scientific Intelligence and coordinated with the United States Army Biological Warfare Laboratories. The program engaged in illegal activities, including the use of U.S. and Canadian citizens as unwitting test

### Embed Documents for ChromaDB

In [9]:
# load it into Chroma with embedded docs, embedding function and directory to contain loaded docs upon persisting.
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
# Ensures loaded document into db will persist in the event of system shutdown/restart
db.persist()

c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Use Chat Model to Multi Query

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(api_key=api_key, temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

In [12]:
# Set logging for the queries made by multi query library imported
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [13]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification of this information?', '2. Can you provide the specific time when this was declassified?', '3. Do you know the exact moment when this information became declassified?']


In [14]:
len(unique_docs)

5

In [15]:
# Get the content of first return instance
print(unique_docs[0].page_content)

The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee's efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.
The most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infiltration of Amer